In [14]:
import sys  
sys.path.insert(0, '../')

In [15]:
import numpy as np
np.set_printoptions(precision=4)

from eomee.tools import hartreefock_rdms, spinize, antisymmetrize, pickpositiveeig, from_unrestricted
from src.scripts.script_erpa import make_doci_hamiltonian

import pyci

In [16]:
def solve(A, B, tol=1.e-8):
    w, v = np.linalg.eigh(B)
    inv_sqrt_w = w ** (-0.5)
    inv_sqrt_w[inv_sqrt_w > 1/tol] = 0.
    ort_m = np.dot(v, np.dot(np.diag(inv_sqrt_w), v.T))
    F_ = np.dot(ort_m.T, np.dot(A, ort_m))
    es, cv = np.linalg.eigh(F_)
    coeffs = np.dot(ort_m, cv)
    return es, coeffs.T


wfntype = {'doci': pyci.doci_wfn, 'fci': pyci.fullci_wfn, }

In [17]:
test2 = ('be/be_sto-6g', 'STO-6G', (2,2)) # ('Be_0_1/Be_0_1_3-21g', '3-21G', (2,2))#
fname, basis, nparts = test2
ref = 'doci'

dirname = f'../data/interim/integrals/{basis}'
h_ij = np.load(f"{dirname}/{fname}_oneint.npy")
g_ijkl = np.load(f"{dirname}/{fname}_twoint.npy")

In [18]:
ham = pyci.hamiltonian(0., h_ij, g_ijkl)
if ref == 'hf':
    wfn = pyci.fullci_wfn(ham.nbasis, *nparts)
    wfn.add_hartreefock_det()
elif ref in ['doci', 'fci']:
    wfn = wfntype[ref](ham.nbasis, *nparts)
    wfn.add_all_dets()
else:
    raise ValueError('wrong wavefunction type.')
op = pyci.sparse_op(ham, wfn)
ev, cv = op.solve(n=1, tol=1.0e-9)
d1, d2 = pyci.compute_rdms(wfn, cv[0])
rdm1, rdm2 = pyci.spinize_rdms(d1, d2)

In [19]:
if ref in ['hf', 'fci']:
    h = spinize(h_ij)
    g = spinize(g_ijkl)
    v = antisymmetrize(g)
if ref == 'doci':
    onemo, twomo = make_doci_hamiltonian(h_ij, g_ijkl)
    h = spinize(onemo)
    g = spinize(twomo)
    v = antisymmetrize(g)
gamma = rdm1
Gamma_ = rdm2

In [20]:
# F_pq = \sum_r h_pr gamma_rq + 2 \sum_rst <rs||tp> Gamma_rstq
F = 0.5 * np.einsum('rstp, rstq-> pq', v, Gamma_)
F += np.dot(h, gamma)

In [21]:
ip, cs = solve(-F, gamma) # 
pw, pcs, _ = pickpositiveeig(ip, cs, tol=1.e-5)
print(ip)

[0.3045 0.3045 0.4482 0.4482 0.4482 0.4482 0.4482 0.4482 4.5406 4.5406]


In [22]:
metric = gamma
ovlp = np.dot(pcs, np.dot(metric, pcs.T))

# print('Diagonal elements of overlap matrix\n', np.diag(ovlp))
print(np.diag(ovlp))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [23]:
def reconstruct_dm1(coeffs, metric):
    tdms = np.einsum('nj,ij->ni', coeffs, metric)
    # print(tdms)
    gamma = np.zeros_like(metric)
    for tdm in tdms:
        gamma += np.einsum('p,q->pq', tdm, tdm)
    return gamma

new_gamma = reconstruct_dm1(pcs, metric)
print(np.trace(new_gamma))

4.000000000000001


In [24]:
k = h_ij.shape[0]
assert np.allclose(new_gamma[:k, :k], new_gamma[k:, k:])

In [25]:
assert np.allclose(F, F.T)
# n = h.shape[0]
# matrix = h @ gamma
# # assert np.allclose(matrix, matrix.T)
# print(matrix[:k,:k])